# We explore this notebook and see how we can make modification to complete our tasks

Some ideas


*   Cross-gender identification
* Age identification
*   Deployment to mobile devices

Dataset provided in assignment brief: https://susanqq.github.io/UTKFace/

Download this: https://drive.google.com/drive/folders/0BxYys69jI14kU0I1YUQyY1ZDRUE?resourcekey=0-01Pth1hq20K4kuGVkp3oBw



# **Age and Gender Estimation in TensorFlow ( Workbook 2, Gender Classification )**

* For Age Estimation, go to [Workbook 1 ( Age Estimation )](https://colab.research.google.com/drive/1to2iolQGIVZgXFWRmRKZGrny5kfXk40h?usp=sharing)

In this notebook, we train a Keras model to classify the gender of a person, given a *face-cropped* image. We use the famous [UTKFace Dataset](https://susanqq.github.io/UTKFace/), which contains 23K images where each image is labelled with its gender, age and ethinicity.


> **Note: Please make sure that you are connected to the GPU runtime of Google Colab. Else, the training might take a decade long. Go to Runtime > Change runtime type > Hardware accelerator.**

## 1) **Import libraries**

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 2) **Processing the data**

Once we've downloaded the dataset, we need to perform the following operations on the dataset, so that it can be used for training our model,

* Reading the image files as 3D NumPy arrays. Note, we'll use 3-channeled RGB images for training the model, so each array will have a shape of `[ img_width , img_height , 3 ]`.

* Split the filename so as to parse the gender of the person in corresponding image. We use the `tf.strings.split()` method for performing this task.

* We one-hot encode the gender, as we'll perform *a two-class* classification.

Once this operations have been performed, we are left with $N$ samples where each sample consists of image array `[ 128 , 128 , 3 ]` and its corresponding label ( one-hot encoded ), the gender of that person, which has a shape `[ 1 , 2 ]`

We'll use `tf.data.Dataset` as it helps us to process the data faster, taking advantage of parallel computing. The above two operations will be mapped on each filename using `tf.data.Dataset.map` method.

In [26]:
# Image size for our model.
MODEL_INPUT_IMAGE_SIZE = [ 128 , 128 ]

TRAIN_TEST_SPLIT = 0.3

# Number of samples to take from dataset
NUM_SAMPLES = 20000

# Trick to one-hot encode the label.
y1 = tf.constant( [ 1. , 0. ] , dtype='float32' ) 
y2 = tf.constant( [ 0. , 1. ] , dtype='float32' ) 

# This method will be mapped for each filename in `list_ds`. 
def parse_image(filename):

    # Read the image from the filename and resize it.
    image_raw = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_raw ,channels=3) 
    image = tf.image.resize(image ,MODEL_INPUT_IMAGE_SIZE) / 255

    # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split( tf.strings.split( filename , '/' )[ 2 ] , '_' )

    # One-hot encode the label
    gender = tf.strings.to_number(parts[1])
    gender = ( gender * y2 ) + ( ( 1 - gender ) * y1 )
    print(gender)

    return image, 2

# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files( './UTKDataset/UTKFace/*' , shuffle=True )
# Map `parse_image` method to all filenames.
dataset = list_ds.map( parse_image , num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
# dataset = list_ds.map(lambda x: tf.py_function(func=parse_image, inp=[x], Tout=(tf.float32, tf.float32)),num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
dataset = dataset.take( NUM_SAMPLES )
print(dataset)

# for e in dataset:
#     print(e)

# Fraction of the dataset to be used for testing.
# split = 0.3

# batch_size = 128

# dir = './UTKDataset/UTKFace/'
# image_data = []

# for i in os.listdir(dir):
#     file = i.split('_')
#     if ((file[0].isnumeric()) and (file[1].isnumeric()) and (file[2].isnumeric())):
#         image_data.append([file[1],i])

# data = pd.DataFrame(image_data, columns=["Gender","Filepath"])

# # data['Age'] = data.Age.astype('float')
# data['Gender'] = data.Gender.astype('float')
# # data['Racevalues'] = data['Racevalues'].astype('float')
# data['Filepath'] = data.Filepath.astype('string')

# # 1-hot embedding
# # data['Age'] = tf.keras.utils.to_categorical(data['Age'])
# data['Gender'] = tf.keras.utils.to_categorical(data['Gender'])
# # data['Racevalues'] = tf.keras.utils.to_categorical(data['Racevalues'])

# # train test split
# train, test = train_test_split(data, test_size = split)
# train.head(10)


Tensor("add:0", shape=(2,), dtype=float32)
<TakeDataset shapes: ((128, 128, 3), ()), types: (tf.float32, tf.int32)>


In [3]:

# testdatagenerator = ImageDataGenerator(rescale=1. /255)
# testdata = testdatagenerator.flow_from_dataframe(dataframe=test,directory=dir,x_col="Filepath",y_col=["Age","Gender","Racevalues"],class_mode="multi_output", batch_size=batch_size)

# traindatagenerator = ImageDataGenerator(rescale=1. /255,shear_range =0.2,zoom_range=0.2,horizontal_flip =True)
# traindata = traindatagenerator.flow_from_dataframe(dataframe=train,directory=dir,x_col="Filepath",y_col=["Age","Gender","Racevalues"],class_mode="multi_output", batch_size=batch_size)


Found 7112 validated image filenames.
Found 16593 validated image filenames.


In [27]:
# Create train and test splits of the dataset.
num_examples_in_test_ds = int( dataset.cardinality().numpy() * TRAIN_TEST_SPLIT )

test_ds = dataset.take( num_examples_in_test_ds )
train_ds = dataset.skip( num_examples_in_test_ds )

print( 'Num examples in train ds {}'.format( train_ds.cardinality() ) )
print( 'Num examples in test ds {}'.format( test_ds.cardinality() ) )


Num examples in train ds 14000
Num examples in test ds 6000



## 3) Model

Our aim is to develop a model which has lesser parameters ( which implies lesser inference time and size ) but powerful enough so that it can generalize better.

* The model takes in a batch of shape `[ None , 128 , 128 , 3 ]` and performs a number of convolutions on it as determined by `num_blocks`.
* Each block consists of a sequence of layers : `Conv2D -> BatchNorm -> LeakyReLU`

```
# Define the conv block.
if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides 
                                            , use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
```

* If `lite_model` is set to `True`, we use [Separable Convolutions](https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728) which have lesser parameters. We could achieve a *faster* model, compromising its performance.

* We stack such`num_blocks` blocks sequentially, where the no. of filters for each layer is taken from `num_filters`.

* Next we add a number of `Dense` layers to learn the features extracted by convolutional layers. Note, we also add a `Dropout` layer, to reduce overfitting. The `rate` for each `Dropout` layer is decreased subsequently for each layer, so that the learnability of `Dense` layer with lesser units ( neurons ) is not affected.

```
def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x
```

* The last `Dense` layer applies the softmax activation function which yields a probability distribution for the two classes `male` and `female`.

> See [this](https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/) blog for choosing the weight decay values used in the above two blocks.

* 👉🏻 The output of the model is a tensor with shape `[ None, 2 ]`


In [14]:
# Custom preprocessing layer to modify image brightness randomly.
class RandomBrightness( tf.keras.layers.Layer ):

    def __init__( self , max_delta ):
        super( RandomBrightness , self ).__init__()
        self.__max_delta = max_delta

    def call( self , inputs ):
        return tf.image.random_brightness( inputs , self.__max_delta )

    def get_config( self ):
        return { "max_delta" : self.__max_delta }


In [23]:
MODEL_INPUT_IMAGE_SIZE = [128, 128]
# Negative slope coefficient for LeakyReLU.
leaky_relu_alpha = 0.2

lite_model = True

# Define the conv block.
def conv( x , num_filters , kernel_size=( 3 , 3 ) , strides=1 ):
    if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides, 
                                            use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
    return x

def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x


# No. of convolution layers to be added.
num_blocks = 5
# Num filters for each conv layer.
num_filters = [ 16 , 32 , 64 , 128 , 256 , 256 ]
# Kernel sizes for each conv layer.
kernel_sizes = [ 3 , 3 , 3 , 3 , 3 , 3 ]

# Init a Input Layer.
inputs = tf.keras.layers.Input( shape=MODEL_INPUT_IMAGE_SIZE + [ 3 ] )

# Add conv blocks sequentially
x = inputs
for i in range( num_blocks ):
    x = conv( x , num_filters=num_filters[ i ] , kernel_size=kernel_sizes[ i ] )
    x = tf.keras.layers.MaxPooling2D()( x )

# Flatten the output of the last Conv layer.
x = tf.keras.layers.Flatten()( x )
conv_output = x 

# Add Dense layers ( Dense -> LeakyReLU -> Dropout )
x = dense( conv_output , 256 , 0.6 )
x = dense( x , 64 , 0.4 )
x = dense( x , 32 , 0.2 )

# age = tf.keras.layers.Dense(2, activation='softmax', name='age')(x)
gender = tf.keras.layers.Dense(2, activation='sigmoid', name='gender')(x)
# race = tf.keras.layers.Dense(2, activation='softmax', name='race')(x)

# Build the Model
model = tf.keras.models.Model(inputs , gender)

# Uncomment the below to view the summary of the model.
model.summary()
# tf.keras.utils.plot_model( model , to_file='architecture.png' )


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
separable_conv2d_5 (Separabl (None, 126, 126, 16)      75        
_________________________________________________________________
batch_normalization_5 (Batch (None, 126, 126, 16)      64        
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 126, 126, 16)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 16)        0         
_________________________________________________________________
separable_conv2d_6 (Separabl (None, 61, 61, 32)        656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 61, 61, 32)        128 


Run this cell to visualize the training of the model in TensorBoard ( in this notebook itself ).

In [5]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/

Reusing TensorBoard on port 6006 (pid 22272), started 12:35:25 ago. (Use '!kill 22272' to kill it.)

## 4) **Compiling the model ( and other callbacks )** 🧱

Once we've defined the architecture for our model, we'll compile our Keras model and also initialize some useful callbacks.

* As we're performing classification, we'll use the Categorical Crossentropy loss function. See [`tf.keras.losses.CategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) for more details.

* We'll use the Adam optimizer for training our model. See [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) for more details.

* For evaluating the performance of our model, we measure the accuracy of our model. See [`tf.keras.metrics.Accuracy`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy) for more details.


#### Callbacks:

* [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to save the Keras model as an H5 file after every epoch.

* [`tf.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) to visualize the training with TensorBoard.

* [`tf.keras.callbacks.EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to stop the training when the evaluation metric i.e the MAE stops improving on the test dataset.

In [28]:
learning_rate = 0.0001
num_epochs = 10 
batch_size = 2

train_ds = train_ds.batch(batch_size).repeat(num_epochs)
test_ds = test_ds.batch(batch_size).repeat(num_epochs)

save_dir = 'checkpoints/cp.ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( save_dir )

logdir = os.path.join( "logs/tb_logs" , datetime.datetime.now().strftime("%Y%m%d-%H%M%S") )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )

early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy' , patience=3 )

model.compile( 
    # loss=tf.keras.losses.sparse_categorical_crossentropy ,
    loss=tf.keras.losses.binary_crossentropy,
    # loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam( learning_rate ) ,
    metrics =[ 'accuracy' ]
)


print(train_ds)

<RepeatDataset shapes: ((None, 128, 128, 3), (None,)), types: (tf.float32, tf.int32)>


## 5) **Train and Evaluate the Model** 🏋🏻‍♂️

Start the training loop with all callbacks packed in.


In [29]:
model.fit( 
    train_ds,
    epochs=num_epochs,  
    validation_data=test_ds,
    callbacks=[checkpoint_callback , tensorboard_callback , early_stopping_callback]
)


Epoch 1/10


ValueError: in user code:

    D:\anaconda3\envs\MDP\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    D:\anaconda3\envs\MDP\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\anaconda3\envs\MDP\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\anaconda3\envs\MDP\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\anaconda3\envs\MDP\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\anaconda3\envs\MDP\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    D:\anaconda3\envs\MDP\lib\si

Evaluate the Model.

In [ ]:

p = model.evaluate( test_ds )
print( 'loss is {} \n accuracy is {} %'.format( p[0] , p[1] * 100 ) )



Save the Keras model to the local disk, so that we can resume training if needed.



In [ ]:

model_name = 'model_gender_augmented' #@param {type: "string"}
model_name_ = model_name + '.h5'

model.save( model_name_ )
files.download( model_name_ ) 



## 6) **Visualize the results**

We'll predict the age from some images taken from `test_ds` and plot them using `matplotlib`.


In [ ]:

fig = plt.figure( figsize=( 12 , 15 ) )
classes = [ 'Male' , 'Female' ]
rows = 5
columns = 2

i = 1
for image , label in test_ds.unbatch().take( 10 ):
    image = image.numpy()
    fig.add_subplot( rows , columns , i )
    plt.imshow( image )
    label_ = classes[ np.argmax( model.predict( np.expand_dims( image , 0 ) ) ) ]
    plt.axis( 'off' )
    plt.title( 'Predicted gender : {} , actual gender : {}'.format( label_ , classes[ np.argmax( label ) ] ) )
    i += 1


 
## 7) **Convert to TensorFlow Lite format** 📡

Our model is to be deployed in an Android app, where we'll use [TF Lite Android](https://bintray.com/google/tensorflow/tensorflow-lite) package to parse the model and make predictions.

We use the `TFLiteConverter` API to convert our Keras Model ( `.h5` ) to a TF Lite buffer ( `.tflite` ). See the [official docs](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter/). We'll produce two TF Lite buffers, one with float16 quantization and other non-quantized model.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_types = [ tf.float16 ]
buffer = converter.convert()

open( '{}_q.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_q.tflite'.format( model_name ) )



For conversion to a non-quantized TF Lite buffer.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
buffer = converter.convert()

open( '{}_nonq.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_nonq.tflite'.format( model_name ) )



## Utility Methods

Use these methods to automate some of the tasks.


In [ ]:

#@title Utility to zip and download a directory
#@markdown Use this method to zip and download a directory. For ex. a TB logs 
#@markdown directory or a checkpoint(s) directory.

dir_to_zip = 'tb_logs' #@param {type: "string"}
output_filename = 'logs.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )


In [ ]:

#@title Utility to delete a directory
#@markdown Use this method to delete a directory. 

dir_path = ''  #@param {type: "string"}
os.system( f'rm -r {dir_path}')
